<a href="https://colab.research.google.com/github/bradleymclellan/stc510/blob/main/Python_Transformations_Essentials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script transforms data on two datasets: crime data from the Phoenix Open Data portal, and weather data from a local file. The crime data is first filtered to remove unnecessary columns, and then the date column is reformatted to match the local weather dataset. The two datasets are merged and filtered to include only data from 2022 onwards.

Next, several plots are created to represent the merged data visually. A bar plot shows the number of crime occurrences based on temperature ranges. A treemap is used to show the number of crime occurrences within each temperature range. A scatter plot is used to show the correlation between violent and non-violent crimes and temperature. Another scatter plot is used to show the grid locations of crimes and their respective temperatures. Finally, a distribution plot is used to show the temperature distributions of different types of crimes.

Finally, the merged data is saved to a CSV file to be used in further analysis.

In [ ]:
import pandas as pd
import requests
import io
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import numpy as np
from datetime import datetime

In [ ]:
# Load Crime data
url_crime = 'https://www.phoenixopendata.com/dataset/cc08aace-9ca9-467f-b6c1-f0879ab1a358/resource/0ce3411a-2fc6-4302-a33f-167f68608a20/download/crimestat.csv'
req = requests.get(url_crime)
crime_df = pd.read_csv(io.StringIO(req.text))
crime_df.drop(columns=['OCCURRED TO', '100 BLOCK ADDR'], inplace=True)

In [ ]:
# Load weather data
weather_df = pd.read_csv('/content/sample_data/crime_weather.csv')

In [ ]:
# Reformat date column to datetime 
crime_df["OCCURRED ON"] = (crime_df["OCCURRED ON"].astype(str).str.split(" ").str[0])
crime_df["OCCURRED ON"] = pd.to_datetime(crime_df["OCCURRED ON"],format="%m/%d/%Y",errors="coerce")
crime_df["OCCURRED ON"] = crime_df["OCCURRED ON"].dt.strftime("%Y-%m-%d")

In [ ]:
# Merge the crime and weather data and filter the data to the year 2022 forward
merged_df = pd.merge(crime_df, weather_df, left_on='OCCURRED ON', right_on='Date', how='left')
merged_df = merged_df[(merged_df['OCCURRED ON'] >= '2022-01-01') & (merged_df['OCCURRED ON'] <= '2023-02-06')]

In [ ]:
# Bar plot of crime occurrences based on temperature
bins = [30,40,50,60,70,80,90,100]
labels = ['< 40', '40-50', '50-60', '60-70', '70-80', '80-90', '> 90']
merged_df['Temperatures'] = pd.cut(merged_df['Temp Max (degrees F)'], bins=bins, labels=labels)
temp_crime = merged_df.groupby(['Temperatures', 'UCR CRIME CATEGORY']).size().reset_index(name='counts')
fig = px.bar(temp_crime, x='UCR CRIME CATEGORY', y='counts', color='Temperatures', barmode='group', height=600)
fig.show()

In [ ]:
# Treemap of crime ocurrances based on temperature 
crime_counts = merged_df.groupby(['UCR CRIME CATEGORY', 'Temp Max (degrees F)']).size().reset_index(name='counts') 
fig = px.treemap(crime_counts, path=['UCR CRIME CATEGORY'], values='counts', color='Temp Max (degrees F)', color_continuous_scale="reds")
fig.show()

In [ ]:
# Grouping crimes by violent and non-violent and the correlation with temperature
crime_cat = {'VIOLENT': ['ROBBERY','ASSAULT','HOMICIDE'],
'NON-VIOLENT': ['LARCENY/THEFT','BURGLARY','VEHICLE THEFT','ARSON']}
merged_df['Violence'] = np.select([merged_df['UCR CRIME CATEGORY'].isin(crime_cat['VIOLENT'])],['VIOLENT'],'NON-VIOLENT')
violence_df = merged_df.groupby(['Violence','Temp Max (degrees F)']).size().reset_index(name='counts')
fig = px.scatter(violence_df, x='Temp Max (degrees F)', y='counts', color='Violence', log_y=True)
fig.show()

In [ ]:
# Scatter plot of grid location and crimes with color hue for temperature
fig = px.scatter(merged_df, x='GRID', y='UCR CRIME CATEGORY', color='Temp Max (degrees F)',size='Temp Max (degrees F)', height=600)
fig.show()

In [ ]:
# Distribution Plot of temperature and types of crime
g = sns.FacetGrid(merged_df, col='UCR CRIME CATEGORY')
g = g.map(sns.histplot, "Temp Max (degrees F)")
plt.show()

In [ ]:
# Save merged dataframe to csv
merged_df.to_csv('/content/sample_data/merged_df.csv', index=False)